In [1]:
import gradio as gr
import os
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, PegasusConfig
from numba import cuda

/home/greg/miniconda3/envs/nlp_class/lib/python3.7/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# UPDATE THESE PARAMETERS BEFORE RUNNING
CONFIG_PATH = "./"
CONFIG_FILENAME = "pretrained_config.json"
MODEL_PATH = "./"
MODEL_FILENAME = "pretrained_model.bin"

# Flag to only run on the cpu (Set True for the demo because of the laptop we used)
USE_CPU_ONLY = False

In [3]:
if torch.cuda.is_available:
    torch.cuda.empty_cache()
cuda_device = cuda.get_current_device()
cuda_device.reset()

device = 'cuda' if (torch.cuda.is_available() and not USE_CPU_ONLY) else 'cpu'

tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

config = PegasusConfig.from_json_file(os.path.join(CONFIG_PATH, CONFIG_FILENAME))
model = PegasusForConditionalGeneration(config).to(device)
state_dict = torch.load(os.path.join(MODEL_PATH, MODEL_FILENAME))
model.load_state_dict(state_dict)

def process(src):
    tokenized_src = tokenizer(src,return_tensors="pt")["input_ids"]
    tokenized_summary = model.generate(tokenized_src)
    summary = tokenizer.batch_decode(tokenized_summary, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return summary[0]

demo = gr.Interface(fn=process, inputs="text", outputs="text", allow_flagging = False)
demo.launch()

/home/greg/miniconda3/envs/nlp_class/lib/python3.7/site-packages/gradio/interface.py:383: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  "The `allow_flagging` parameter in `Interface` now"


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<fastapi.applications.FastAPI at 0x7f41bc3c0d50>,
 'http://127.0.0.1:7860/',
 None)